In [6]:
! pip install transformers peft accelerate evaluate
! pip install --upgrade datasets

  Using cached datasets-3.6.0-py3-none-any.whl.metadata (19 kB)
  Using cached fsspec-2025.3.0-py3-none-any.whl.metadata (11 kB)
Using cached datasets-3.6.0-py3-none-any.whl (491 kB)
Using cached fsspec-2025.3.0-py3-none-any.whl (193 kB)
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
  Attempting uninstall: datasets
    Found existing installation: datasets 2.14.4
    Uninstalling datasets-2.14.4:
      Successfully uninstalled datasets-2.14.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is incompatible.


In [5]:
from huggingface_hub import notebook_login

notebook_login()

In [1]:
import os
import torch
import peft
import evaluate
from torch.optim import AdamW
from torch.utils.data import DataLoader
from datasets import load_dataset
from tqdm import tqdm
from transformers import AutoModelForSequenceClassification, AutoTokenizer,get_linear_schedule_with_warmup,set_seed

In [2]:
batch_size=8
model_name='roberta-large'
task='mrpc'
peft_type=peft.PeftType.IA3
device='cuda'
num_epochs=12

In [3]:
# Setting Peft Configuration for IA3
peft_config=peft.IA3Config(task_type='SEQ_CLS',inference_mode=False)
lr=1e-3
# task-type=SequenceClassification

In [6]:
if any(k in model_name for k in ("gpt", "opt", "bloom")):
    padding_side = "left"
else:
    padding_side = "right"

tokenizer = AutoTokenizer.from_pretrained(model_name, padding_side=padding_side)
if getattr(tokenizer, "pad_token_id") is None:
    tokenizer.pad_token_id = tokenizer.eos_token_id

datasets = load_dataset("glue",'mrpc')
metric = evaluate.load("glue",'mrpc')


def tokenize_function(examples):
    # max_length=None => use the model max length (it's actually the default)
    outputs = tokenizer(examples["sentence1"], examples["sentence2"], truncation=True, max_length=None)
    return outputs


tokenized_datasets = datasets.map(
    tokenize_function,
    batched=True,
    remove_columns=["idx", "sentence1", "sentence2"],
)

# We also rename the 'label' column to 'labels' which is the expected name for labels by the models of the
# transformers library
tokenized_datasets = tokenized_datasets.rename_column("label", "labels")


def collate_fn(examples):
    return tokenizer.pad(examples, padding="longest", return_tensors="pt")


# Instantiate dataloaders.
train_dataloader = DataLoader(tokenized_datasets["train"], shuffle=True, collate_fn=collate_fn, batch_size=batch_size)
eval_dataloader = DataLoader(
    tokenized_datasets["validation"], shuffle=False, collate_fn=collate_fn, batch_size=batch_size
)
test_dataloader = DataLoader(tokenized_datasets["test"], shuffle=False, collate_fn=collate_fn, batch_size=batch_size)

train-00000-of-00001.parquet:   0%|          | 0.00/649k [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/75.7k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/308k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/3668 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/408 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1725 [00:00<?, ? examples/s]

Map:   0%|          | 0/3668 [00:00<?, ? examples/s]

Map:   0%|          | 0/408 [00:00<?, ? examples/s]

Map:   0%|          | 0/1725 [00:00<?, ? examples/s]

In [7]:
model=AutoModelForSequenceClassification.from_pretrained(model_name,return_dict=True)
model=peft.get_peft_model(model,peft_config)
model.print_trainable_parameters()
model

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-large and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


trainable params: 1,223,682 || all params: 356,585,476 || trainable%: 0.3432


PeftModelForSequenceClassification(
  (base_model): IA3Model(
    (model): RobertaForSequenceClassification(
      (roberta): RobertaModel(
        (embeddings): RobertaEmbeddings(
          (word_embeddings): Embedding(50265, 1024, padding_idx=1)
          (position_embeddings): Embedding(514, 1024, padding_idx=1)
          (token_type_embeddings): Embedding(1, 1024)
          (LayerNorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (encoder): RobertaEncoder(
          (layer): ModuleList(
            (0-23): 24 x RobertaLayer(
              (attention): RobertaAttention(
                (self): RobertaSdpaSelfAttention(
                  (query): Linear(in_features=1024, out_features=1024, bias=True)
                  (key): Linear(
                    (base_layer): Linear(in_features=1024, out_features=1024, bias=True)
                    (ia3_l): ParameterDict(  (default): Parameter containing: [torch.F

In [8]:
optimizer=AdamW(params=model.parameters(),lr=lr)

# Lr Schedular
lr_scheduler=get_linear_schedule_with_warmup(
    optimizer=optimizer,
    num_warmup_steps= 0.06 * (len(train_dataloader)*num_epochs),
    num_training_steps=(len(train_dataloader) * num_epochs)
)

In [9]:
model.to(device)

for epoch in range(num_epochs):
  model.train()
  for step, batch in enumerate(tqdm(train_dataloader)):
    batch.to(device)
    outputs=model(**batch)
    loss=outputs.loss
    loss.backward()
    optimizer.step()
    lr_scheduler.step()
    optimizer.zero_grad()

  model.eval()
  for step, batch in enumerate(tqdm(eval_dataloader)):
    batch.to(device)
    with torch.no_grad():
      outputs=model(**batch)
    predictions=outputs.logits.argmax(dim=-1)
    predictions,references=predictions, batch['labels']
    metric.add_batch(
        predictions=predictions,
        references=references
    )
  eval_metric=metric.compute()
  print(f"epoch {epoch}:",eval_metric)

100%|██████████| 51/51 [00:05<00:00,  9.35it/s]


epoch 0: {'accuracy': 0.7132352941176471, 'f1': 0.821917808219178}


100%|██████████| 51/51 [00:05<00:00,  9.34it/s]


epoch 1: {'accuracy': 0.7696078431372549, 'f1': 0.8041666666666667}


100%|██████████| 51/51 [00:05<00:00,  9.32it/s]


...

epoch 9: {'accuracy': 0.8823529411764706, 'f1': 0.9124087591240876}


100%|██████████| 51/51 [00:05<00:00,  9.22it/s]


epoch 10: {'accuracy': 0.8700980392156863, 'f1': 0.9065255731922398}


100%|██████████| 51/51 [00:05<00:00,  9.24it/s]

epoch 11: {'accuracy': 0.875, 'f1': 0.9084380610412927}


In [10]:
model.push_to_hub('OmBhandwalkar/roberta-large-peft-ia3',use_auth_tokens=True)

adapter_model.safetensors:   0%|          | 0.00/4.91M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/OmBhandwalkar/roberta-large-peft-ia3/commit/5937f06afb05aabddd3a3e680250b273a6cc0f33', commit_message='Upload model', commit_description='', oid='5937f06afb05aabddd3a3e680250b273a6cc0f33', pr_url=None, repo_url=RepoUrl('https://huggingface.co/OmBhandwalkar/roberta-large-peft-ia3', endpoint='https://huggingface.co', repo_type='model', repo_id='OmBhandwalkar/roberta-large-peft-ia3'), pr_revision=None, pr_num=None)

## Load Adapter from HUB

In [13]:
import torch
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer

peft_model_id = "OmBhandwalkar/roberta-large-peft-ia3"
config = PeftConfig.from_pretrained(peft_model_id)
inference_model = AutoModelForSequenceClassification.from_pretrained(config.base_model_name_or_path)
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)

# Load the Lora model
inference_model = PeftModel.from_pretrained(inference_model, peft_model_id)

inference_model.to(device)
inference_model.eval()
for step, batch in enumerate(tqdm(eval_dataloader)):
    batch.to(device)
    with torch.no_grad():
        outputs = inference_model(**batch)
    predictions = outputs.logits.argmax(dim=-1)
    predictions, references = predictions, batch["labels"]
    metric.add_batch(
        predictions=predictions,
        references=references,
    )

eval_metric = metric.compute()
print(eval_metric)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-large and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


adapter_model.safetensors:   0%|          | 0.00/4.91M [00:00<?, ?B/s]

100%|██████████| 51/51 [00:05<00:00,  9.11it/s]

{'accuracy': 0.875, 'f1': 0.9084380610412927}
